# Recap train at scale

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# First, check that you run this notebook with the correct taxifare-env kernel
import taxifare
taxifare.__file__

'/home/unix_blamb/code/blamb888/data-train-in-the-cloud/taxifare/__init__.py'

In [3]:
# You should be able to load the following files
import os
from taxifare.params import *
data_processed_path_200k = os.path.join(LOCAL_DATA_PATH, "processed","processed_2009-01-01_2015-01-01_200k.csv")
data_processed_path_all = os.path.join(LOCAL_DATA_PATH, "processed","processed_2009-01-01_2015-01-01_all.csv")

<details>
    <summary markdown='span'>If files are missings</summary>

```bash
make reset_local_files_with_csv_solutions
```

# 1) Explain concepts of incremental fit by chunks

<img src='https://wagon-public-datasets.s3.amazonaws.com/data-science-images/07-ML-OPS/train_by_chunk.png'>

# 2) Explain code solution for `main_local.train()`

```python
def train(min_date:str = '2009-01-01', max_date:str = '2015-01-01') -> None:
    """
    Incremental train on the (already preprocessed) dataset locally stored.
    - Loading data chunk-by-chunk
    - Updating the weight of the model for each chunk
    - Saving validation metrics at each chunks, and final model weights on local disk
    """
    # ...
```

Let's launch a training by batch on 200k rows! (set DATA_SIZE='200k' in params.py)

In [4]:
from taxifare.interface.main_local import train
train()

2023-07-21 18:49:19.263185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 18:49:19.396597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-21 18:49:19.396621: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-21 18:49:19.430710: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-21 18:49:20.227702: W tensorflow/stream_executor/platform/de


Loading TensorFlow...

✅ TensorFlow loaded (0.0s)

 ⭐️ Use case: train in batches


ValueError: could not convert string to float: 'fare_amount'

# 3) 💻 Tensorflow tricks to partial fit without manual chunks


**📚Resources📚**
- tf CSV guide: https://www.tensorflow.org/guide/data#consuming_csv_data
- tf CSV tuto: https://www.tensorflow.org/tutorials/load_data/csv
- tf Datasets https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/data.ipynb#scrollTo=x5z5B11UjDTd

**Import packages**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, layers, regularizers
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

**Import model**

We'll copy paste it below to make it more explicit

In [ ]:
def build_model():
    
    reg = regularizers.l1_l2(l2=0.005)

    model = Sequential()
    model.add(layers.Input(shape=(65,)))
    model.add(layers.Dense(100, activation="relu", kernel_regularizer=reg))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.BatchNormalization(momentum=0.9))  # use momentum=0 to only use statistic of the last seen minibatch in inference mode ("short memory"). Use 1 to average statistics of all seen batch during training histories.
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(1, activation="linear"))
    
    optimizer = keras.optimizers.Adam(learning_rate= 0.001)
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mae"])
    
    return model


In [ ]:
es = EarlyStopping(monitor="val_loss",
                       patience=2,
                       restore_best_weights=True,
                       verbose=0)

In [ ]:
BATCH_SIZE=265

## 3.1) If data fit in memory 😇

In [ ]:
df_small = pd.read_csv(data_processed_path_200k, header=None)
df_small

In [ ]:
features = df_small.drop(columns=[65]).to_numpy()
target = df_small[[65]].to_numpy()

In [ ]:
print(features.shape)
print(target.shape)
n_samples = features.shape[0]
n_features = features.shape[1]

### a) passing numpy arrays

In [ ]:
model = build_model()

model.fit(x=features, y=target, batch_size=BATCH_SIZE, validation_split=0.3, callbacks=[es], epochs=10)

### b) passing `datasets` iterators

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((features, target))
ds = ds.batch(BATCH_SIZE)  # Set batch size

In [ ]:
ds.element_spec

In [ ]:
# First sample: feature_1, target_1
f1, t1 = next(iter(ds))
(f1.shape, t1.shape)

In [ ]:
model = build_model()
model.fit(ds, epochs=5)

## 3.2) If data is too large to fit in memory ? 🧐 

💡 Use `make_csv_dataset` helper

More info on this tutorial https://www.tensorflow.org/tutorials/load_data/csv

In [ ]:
ds = tf.data.experimental.make_csv_dataset(
    data_processed_path_all,
    batch_size=BATCH_SIZE,
    header=False,
    column_names=list(df_small.columns),
    label_name=65,
    num_epochs=1,
    ignore_errors=True)

In [ ]:
ds.element_spec

We can now iterate on our dataset `ds` without ever loading all the CSV in memory!

In [ ]:
feat1, target1 = next(iter(ds))

Let's inspect the first element (feat1, target1)

👇 target1 is simply a 1D tensor that contains BATCH_SIZE prices

In [ ]:
print('target1.shape: ', target1.shape)

👇 feat1 is a bit more complex, it's an ordered dict that contains N_FEAT=65 elements, each being a BATCH_SIZE = 256 1D vector

In [ ]:
print(type(feat1))
print(len(feat1))
print(feat1[0].shape)

Let's rearrange it as a (BATCH_SIZE, N_FEAT) tensor as we are used to manipulate

In [ ]:
def stack(x):
    return tf.stack([x[i] for i in range(65)], axis=1)

stack(feat1).shape

We can now `map` our dataset iterator with this transformation

In [ ]:
ds = ds.map(lambda x,y: (stack(x),y))

In [ ]:
ds.element_spec

And use it directly to train our model on the **full dataset**! 

We can train on TB size CSV without RAM limitation!

In [ ]:
model = build_model()
model.fit(ds, epochs=5)